# Use-case
Encapsulates an operation, allows for queueing operations or undoing them. Also knows as action or transaction.

# Situation when usage is suggested
* Parameterization of objects using actions.
* Creation of queues in programs, managing commands in different process.
* Undoing changes to the data.
* Registering changes done to reapply them if necessary.
* Creation of groups of actions that have to be executed all together or none at all (transactions).

# Structure
![title](img/command.jpg)
By Vanderjoe - Own work, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=62530466

# Implementation


In [113]:
import json


class Database:
    content: dict[str, list]

    def __init__(self):
        with open("db.json", "r+") as db:
            self.content = json.load(db)
            self.id = self.content["data"][-1][0] if self.content["data"] else 0

    def save(self):
        with open("db.json", "w") as db:
            json.dump(self.content ,db)

    def add_record(self, *args):
        if len(args) != len(self.content["header"])-1:
            raise ValueError("Wrong number of values")
        self.id += 1
        self.content["data"].append([self.id, *args])

    def remove_record(self, row_id: int):
        self.content["data"].pop(row_id-1)
        for idx, row in enumerate(self.content["data"], start=1):
            row[0] = idx

    def add_column(self, name, default):
        self.content["header"].append(name)
        for row in self.content["data"]:
            row.append(default)

    def remove_column(self, name):
        if name == "id":
            raise ValueError
        column_id = self.content["header"].index(name)
        self.content["header"].pop(column_id)
        for row in self.content["data"]:
            row.pop(column_id)

    def show(self):
        header_line = " | ".join(f"{header:>10}" for header in self.content["header"])
        lines = [header_line]
        for row in self.content["data"]:
            lines.append(" | ".join(f"{value:>10}" for value in row))
        print("\n".join(lines))


db = Database()
db.show()

        id


In [63]:
from abc import ABC, abstractmethod


class Command(ABC):

    @abstractmethod
    def execute(self, database: Database):
        pass

    @abstractmethod
    def undo(self, database: Database):
        pass


In [137]:
from enum import Enum


class DatabaseCommand(Command):
    class Action(Enum):
        ADD_COLUMN = 0
        REMOVE_COLUMN = 1
        ADD_RECORD = 2
        REMOVE_RECORD = 3

    def __init__(self, action, *args, **kwargs):
        self.action = action
        self.args = args
        self.kwargs = kwargs

    def execute(self, database: Database):
        if self.action == self.Action.ADD_COLUMN:
            database.add_column(*self.args, **self.kwargs)
        if self.action == self.Action.REMOVE_COLUMN:
            database.remove_column(*self.args, **self.kwargs)
        if self.action == self.Action.ADD_RECORD:
            database.add_record(*self.args)
        if self.action == self.Action.REMOVE_RECORD:
            database.remove_record(*self.args, **self.kwargs)

    def undo(self, database: Database):
        if self.action == self.Action.ADD_COLUMN:
            database.remove_column(self.kwargs["name"])
        if self.action == self.Action.REMOVE_COLUMN:
            raise TypeError("Cannot reverse this operation")
        if self.action == self.Action.ADD_RECORD:
            database.remove_record(0)
        if self.action == self.Action.REMOVE_RECORD:
            raise TypeError("Cannot reverse this operation")


class DatabaseCompositeCommand(Command, list):

    def __init__(self, items=None):
        super().__init__()
        items = items or []
        for item in items:
            self.append(item)

    def execute(self, database: Database):
        for command in self:
            command.execute(database)

    def undo(self, database: Database):
        for command in self:
            command.undo(database)


In [131]:
db = Database()
command_1 = DatabaseCommand(DatabaseCommand.Action.ADD_COLUMN, name="name", default="John")
command_2 = DatabaseCommand(DatabaseCommand.Action.ADD_RECORD, "Jane")
command_3 = DatabaseCommand(DatabaseCommand.Action.ADD_RECORD, "Jonny")

command_1.execute(db)
command_2.execute(db)
command_3.execute(db)
db.show()

        id |       name
         1 |      Jonny


In [132]:
db2 = Database()
command_1.execute(db2)
command_2.execute(db2)
command_3.execute(db2)
db2.show()

        id |       name
         1 |       Jane
         2 |      Jonny


In [138]:
db3 = Database()
composite_command = DatabaseCompositeCommand([command_1, command_2, command_3])
composite_command.execute(db3)
db3.show()


        id |       name
         1 |       Jane
         2 |      Jonny


In [139]:
db4 = Database()
composite_command.execute(db4)
db4.show()


        id |       name
         1 |       Jane
         2 |      Jonny
